In [ ]:
import cv2
import face_recognition
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.models as models
from matplotlib import pyplot as plt

In [ ]:


# Create a VideoCapture object to access the camera (usually 0 for the default camera)
cap = cv2.VideoCapture(0)
counter=0
while True:
   
    
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    # Display the frame in a window
    
    cv2.imwrite('dataset/rei/camera_%s.jpg'%counter, frame)   ## number one
    thename='dataset/rei/camera_%s.jpg'%counter
    cv2.imshow('Camera Feed.   press q to quit', frame)
    image = face_recognition.load_image_file(thename)
    counter+=1
    # Press 'q' to exit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



### for adding more data, first go to number one then add new folder of the person (add the folder on computer too.) then run it like usual. also go to modify head and change num classes too

## machine learning model 

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transforms_pred = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder(root='dataset/', transform=data_transforms)


### see data set

In [ ]:
dataset.classes

### load model

In [ ]:
import torchvision.models as models

# Load the pre-trained ResNet-34 model
model = models.resnet34(pretrained=True)


### modify class head 

In [ ]:
import torch.nn as nn
in_features = model.fc.in_features
num_classes = 4  # Number of classes in your dataset     change this if you add more data 

# Replace the last fully connected layer

model.fc = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes),
    nn.Sigmoid()  # For binary classification
)


### loss and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


### training 

In [ ]:


# Define your data transformations, model, loss, and optimizer here

# Set the number of training epochs, batch size, and other hyperparameters
num_epochs = 15
batch_size = 16

# Create data loaders
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 mini-batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], Loss: {running_loss / 10:.4f}")
            running_loss = 0.0

# Save the trained model
torch.save(model.state_dict(), 'fine_tuned_resnet34.pth')


### prediction 

In [ ]:
image = Image.open("dataset/gang/camera_56.jpg")
image2 = transforms_pred(image)
image2 = image2.unsqueeze(0)  # Add a batch dimension

# Ensure that your model is in evaluation mode
model.eval()
with torch.no_grad():
    output = model(image2)
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)
    predicted_label=dataset.classes[predicted_class[0].item()]

    # Add a label
    plt.title("%s"%predicted_label)
    plt.imshow(image)

    # Show the image with the label
    plt.show()


### display in video 

In [ ]:
# Create a VideoCapture object to access the camera (usually 0 for the default camera)
cap = cv2.VideoCapture(0)
counter=0
cv2.destroyAllWindows()
cv2.namedWindow("Video Feed", cv2.WINDOW_NORMAL)
while True:
   
    
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    image2 = transforms_pred(pil_image)
    image2 = image2.unsqueeze(0)  # Add a batch dimension

    # Ensure that your model is in evaluation mode
    model.eval()
    with torch.no_grad():
        output = model(image2)
        probabilities = torch.softmax(output, dim=1)
        _, predicted_class = torch.max(probabilities, 1)
        predicted_label=dataset.classes[predicted_class[0].item()]
    
    # Display the frame in a window
    dalabel= predicted_label+"   press q to quit"
    cv2.imshow(dalabel, frame)
    # Press 'q' to exit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
